# Scrap them all !

In [12]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request, response


In [5]:
%qtconsole

In [42]:
def hasNumbers(inputString):
     return any(char.isdigit() for char in inputString)

def clean_ministres_wiki(wiki_ministre) : 
    wiki_ministres_clean = [x for x in wiki_ministre if ('France' not in x) & 
                            ('jpg' not in x) & ('Fichier' not in x) & (len(x) <= 35) &
                            ('gauche' not in x) & ('Outre-' not in x) & ('Ville' not in (x)) &
                            ('Logement' not in x) & ('Budget' not in x)  & ('Walwari' not in x) &
                            ('#cite' not in x)]
    print('%s ministres trouvés' % (len(wiki_ministres_clean)))
    
    return wiki_ministres_clean

def get_my_gouv(link_gouv):
    # Request
    header = {'User-Agent': 'Mozilla/5.0'} # Needed to prevent 403 error on Wikipedia
    page = request.Request(link_gouv, headers=header)
    html = request.urlopen(page) 
    soup = BeautifulSoup(html, "lxml")
    cells = soup.findAll("table", {"class": "wikitable sortable"})
    #  Init
    wiki_ministres = []
    fonction_ministres = [] 
    n_ministres = []
    n_min = 0
    # Scraping
    for cell in cells: # Tableau principal
        cells_href = cell.findAll("a", href=True)
        for c_href in cells_href: # éléments de chaque tableau
            wiki_ministres.append(c_href['href'])
            fonction_ministres.append(c_href.get_text())
            n_ministres.append(n_min)
            n_min += 1
    wiki_ministres_clean = clean_ministres_wiki(wiki_ministres)
    return (fonction_ministres, wiki_ministres_clean)

In [43]:
dict_holland = {'holland_ayrault_1': "https://fr.wikipedia.org/wiki/Gouvernement_Jean-Marc_Ayrault_(1)",
                'holland_ayrault_2': "https://fr.wikipedia.org/wiki/Gouvernement_Jean-Marc_Ayrault_(2)",
                'holland_valls_1': "https://fr.wikipedia.org/wiki/Gouvernement_Manuel_Valls_(1)",
                'holland_valls_2': "https://fr.wikipedia.org/wiki/Gouvernement_Manuel_Valls_(2)" }

for keys, values in dict_holland.items():
    gouv = pd.DataFrame()
    (fonction_ministres, wiki_ministres_clean) = get_my_gouv(values)
    gouv[keys] = wiki_ministres_clean
    

169
36 ministres trouvés
153
32 ministres trouvés
152
35 ministres trouvés
184
44 ministres trouvés


In [58]:
df = pd.DataFrame()
df['fonctions_ministres '] = fonction_ministres

In [51]:
fonction_ministres_clean = [x for x in fonction_ministres if ('ministre' in x) or ('Ministre' in x) or ('Secr' in x) & ('[' not in x)]
df = pd.DataFrame()
df['fonction_ministres'] = fonction_ministres
df.head()

,fonction_ministres
0,
1,Premier ministre
2,Manuel Valls
3,PS
4,


In [71]:
fonction_ministres_clean =  [x for x in fonction_ministres if ('[' not in x)]
df = pd.DataFrame()
df['fonction_ministres_clean'] = fonction_ministres_clean
df.fonction_ministres_clean.isin(['PS', 'PRG', 'Sans étiquette', 'EELV', 'Écologistes !', 'Divers écologiste']).value_counts()

False    143
True      39
dtype: int64

In [69]:
fonction_ministres_clean

['',
 'Premier ministre',
 'Manuel Valls',
 'PS',
 '',
 'Ministre des Affaires étrangères et du Développement international',
 'Jean-Marc Ayrault',
 'PS',
 '',
 "Ministre de l'Environnement, de l'Énergie et de la Mer, chargée des relations internationales sur le climat",
 'Ségolène Royal',
 'PS',
 '',
 "Ministre de l'Éducation nationale",
 "de l'Enseignement supérieur et de la Recherche",
 'Najat Vallaud-Belkacem',
 'PS',
 '',
 'Ministre des Finances et des Comptes publics',
 'Michel Sapin',
 'PS',
 '',
 'Ministre des Affaires sociales et de la Santé',
 'Marisol Touraine',
 'PS',
 '',
 'Ministre de la Défense',
 'Jean-Yves Le Drian',
 'PS',
 '',
 'Garde des Sceaux, ministre de la Justice',
 'Jean-Jacques Urvoas',
 'PS',
 '',
 "Ministre du Travail, de l'Emploi, de la Formation professionnelle et du Dialogue social",
 'Myriam El Khomri',
 'PS',
 '',
 "Ministre de l'Aménagement du territoire, de la Ruralité et des Collectivités territoriales",
 'Jean-Michel Baylet',
 'PRG',
 '',
 "Ministr

In [53]:
wiki_ministres_clean

['/wiki/Manuel_Valls',
 '/wiki/Jean-Marc_Ayrault',
 '/wiki/S%C3%A9gol%C3%A8ne_Royal',
 '/wiki/Najat_Vallaud-Belkacem',
 '/wiki/Michel_Sapin',
 '/wiki/Marisol_Touraine',
 '/wiki/Jean-Yves_Le_Drian',
 '/wiki/Jean-Jacques_Urvoas',
 '/wiki/Myriam_El_Khomri',
 '/wiki/Jean-Michel_Baylet',
 '/wiki/Bernard_Cazeneuve',
 '/wiki/St%C3%A9phane_Le_Foll',
 '/wiki/Emmanuelle_Cosse',
 '/wiki/%C3%89cologie_politique',
 '/wiki/Emmanuel_Macron',
 '/wiki/Sans_%C3%A9tiquette',
 '/wiki/Audrey_Azoulay',
 '/wiki/Sans_%C3%A9tiquette',
 '/wiki/Laurence_Rossignol',
 '/wiki/Annick_Girardin',
 '/wiki/Patrick_Kanner',
 '/wiki/George_Pau-Langevin',
 '/wiki/Jean-Marie_Le_Guen',
 '/wiki/Ericka_Bareigts',
 '/wiki/Jean-Vincent_Plac%C3%A9',
 '/wiki/%C3%89cologistes_!',
 '/wiki/Juliette_M%C3%A9adel',
 '/wiki/Harlem_D%C3%A9sir',
 '/wiki/Matthias_Fekl',
 '/wiki/Andr%C3%A9_Vallini',
 '/wiki/Alain_Vidalies',
 '/wiki/Barbara_Pompili',
 '/wiki/%C3%89cologie_politique',
 '/wiki/Thierry_Mandon',
 '/wiki/Christian_Eckert',
 '/wiki

## Looping the ministres

In [19]:
for ministre in link_ministres_clean[:2]:
    page_ministre = "https://fr.wikipedia.org" + ministre
    header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
    page = request.Request(page_ministre, headers=header)
    html = request.urlopen(page)
    soup = BeautifulSoup(html, "lxml")

In [22]:
print(soup.find('table', {'class': 'infobox_v2'}))

<table class="infobox_v2">
<tr>
<td class="entete defaut" colspan="2" style="background-color:#6688FF;color:#FFFFFF">Laurent Fabius</td>
</tr>
<tr>
<td colspan="2" style="text-align:center; line-height: 1.5em;"><a class="image" href="/wiki/Fichier:Laurent_Fabius_January_2015.jpg"><img alt="Laurent Fabius, en janvier 2015." data-file-height="1182" data-file-width="888" height="293" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Laurent_Fabius_January_2015.jpg/220px-Laurent_Fabius_January_2015.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Laurent_Fabius_January_2015.jpg/330px-Laurent_Fabius_January_2015.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Laurent_Fabius_January_2015.jpg/440px-Laurent_Fabius_January_2015.jpg 2x" width="220"/></a><br/>
Laurent Fabius, en janvier 2015.</td>
</tr>
<tr>
<th colspan="2" style="padding:4px;text-align:center;background-color:#6688FF;color:#FFFFFF">Fonctions</th>
</tr>
<tr>
<th colspan="2" style="padding:4px;